In [11]:
from pathlib import Path
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from import_ipynb import *
from script import folders_files

In [12]:
PATH = Path('soccer')
PATH.mkdir(parents=True, exist_ok=True)
FILE = 'script.ipynb'
FILE_PATH = PATH / FILE
FILE_PATH

WindowsPath('soccer/script.ipynb')

In [13]:
print(folders_files('soccer'))

{}
